In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
import pickle

In [ ]:
targets.sum() / targets.shape[0]

In [ ]:
unscaled_inputs = data_with_targets.iloc[:,:-1]

In [ ]:
data_scaler = StandardScaler()

In [ ]:
class CustomScaler(BaseEstimator,TransformerMixin): 
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        self.scaler = StandardScaler(copy,with_mean,with_std)
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    def transform(self, X, y=None, copy=None):
        
        init_col_order = X.columns
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [ ]:
unscaled_inputs.columns.values

In [ ]:
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']

In [ ]:
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [ ]:
data_scaler = CustomScaler(columns_to_scale)

In [ ]:
data_scaler.fit(unscaled_inputs)

In [ ]:
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [ ]:
train_test_split(scaled_inputs, targets)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, 
                                                                            test_size = 0.2, random_state = 20)

In [ ]:
reg = LogisticRegression()

In [ ]:
reg.fit(x_train,y_train)

In [ ]:
reg.score(x_train,y_train)

In [ ]:
model_outputs = reg.predict(x_train)
model_outputs

In [ ]:
model_outputs == y_train

In [ ]:
np.sum((model_outputs==y_train))

In [ ]:
model_outputs.shape[0]

In [ ]:
np.sum((model_outputs==y_train)) / model_outputs.shape[0]

In [ ]:
reg.intercept_

In [ ]:
reg.coef_

In [ ]:
unscaled_inputs.columns.values

In [ ]:
feature_name = unscaled_inputs.columns.values

In [ ]:
summary_table = pd.DataFrame (columns=['Feature name'], data = feature_name)

summary_table['Coefficient'] = np.transpose(reg.coef_)

summary_table

In [ ]:
summary_table.index = summary_table.index + 1

summary_table.loc[0] = ['Intercept', reg.intercept_[0]]

summary_table = summary_table.sort_index()
summary_table

In [ ]:
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)

In [ ]:
summary_table

In [ ]:
summary_table.sort_values('Odds_ratio', ascending=False)

In [ ]:
reg.score(x_test,y_test)

In [ ]:
predicted_proba = reg.predict_proba(x_test)

predicted_proba

In [ ]:
predicted_proba.shape

In [ ]:
predicted_proba[:,1]

In [ ]:
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [ ]:
with open('scaler','wb') as file:
    pickle.dump(data_scaler, file)